In [1]:
REPOSITORY_PATH = 'BoostingCNN-Jpeg-Primary-Quantization-Matrix-Estimation'

import sys
import os
sys.path.append(REPOSITORY_PATH)

In [2]:
import cv2
import numpy as np
import tensorflow as tf
from networks import custom_two_terms_loss_wrapper, custom_softmax_activation, custom_mse_wrapper
from utils import max_min_coefficient, label2coefficient, string2Q

# Playground

In [3]:
def preprocess_input(im_file, target_size, scale=255.):
    im = cv2.imread(im_file)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2YCrCb)[:,:,0]
    if im.shape != target_size:
        im = cv2.resize(im, target_size)

    return im.astype(np.float32) / scale

In [4]:
def load_qf_map(filename, n_coefs=15):
    qf_list = np.load(filename, allow_pickle=True)
    qf_labels = [qf[:2] for qf in qf_list]
    qf_coeffs = [string2Q(qf[2])[:n_coefs] for qf in qf_list]
    return qf_labels, qf_coeffs

In [5]:
def estimation_by_mse(x, y, weights=None):
    diff = (x - y) ** 2
    return np.average(diff, axis=1, weights=weights)

In [6]:
def get_QF(x, y, labels, estimator=estimation_by_mse):
    argmin = np.argmin(estimator(x, y))
    return labels[argmin]

In [7]:
modelname = 'model_QF1_55-98_QF2_80-2-term-loss.h5'
model_file = os.path.join(REPOSITORY_PATH, 'models', modelname)
# img_file = os.path.join('data', 'roxy.jpeg')
img_file = 'BoostingCNN-Jpeg-Primary-Quantization-Matrix-Estimation/resources/00000000_redaf7d93t.TIF_85_90.png'
qf_labels, qf_coeffs = load_qf_map(os.path.join(REPOSITORY_PATH, 'resources', 'qf1_qf2_map_90.npy'))

In [8]:
max_coeffs, min_coeffs = max_min_coefficient(quality_range=(50, 100),
                                             n_coeffs=15,
                                             zig_zag_order=True)

In [9]:
model = tf.keras.models.load_model(model_file,
                                    custom_objects=({'custom_softmax': custom_softmax_activation(max_coeffs),
                                                    'custom_two_terms_loss_wrapper': custom_two_terms_loss_wrapper(
                                                        max_coeffs, 0.8),
                                                    'custom_mse': custom_mse_wrapper(max_coeffs)})
                                    )

In [10]:
x = preprocess_input(img_file, (64, 64), 255.)
prediction = model.predict(np.expand_dims(x, [0, -1]))

predicted_label = label2coefficient(prediction.flatten(), max_coefficients=max_coeffs)
print(predicted_label)

1/1 [==============================] - 0s 393ms/step
[1 1 1 1 1 1 1 1 1 1 1 1 1 2 1]


In [11]:
get_QF(qf_coeffs, predicted_label, qf_labels, estimator=estimation_by_mse)

array([96, 90], dtype=object)

# Download raw images from RAISE1k

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/RAISE_1k.csv')
df.head()

,File,NEF,TIFF,Date Created,Date Modified,File Size,Image Size,Date Shot,Time Zone and Date,Image Quality,...,Sharpening.1,Latitude,Longitude,Altitude,Altitude Reference,Heading,UTC,Map Datum,Scene Mode,Keywords
0,r000da54ft,http://193.205.194.113/RAISE/NEF/r000da54ft.NEF,http://193.205.194.113/RAISE/TIFF/r000da54ft.TIF,9/20/2014 23:51,3/19/2012 9:14,9.79 MB,L (4288 x 2848),14:14.0,"UTC+1, DST:ON",Compressed RAW (12-bit),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nature; outdoor
1,r001d260dt,http://193.205.194.113/RAISE/NEF/r001d260dt.NEF,http://193.205.194.113/RAISE/TIFF/r001d260dt.TIF,9/21/2014 9:58,5/11/2013 6:45,20.0 MB,L (4928 x 3264),45:58.6,"UTC+1, DST:ON",Lossless Compressed RAW (14-bit),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aperture Priority,buildings; outdoor
2,r002fc3e2t,http://193.205.194.113/RAISE/NEF/r002fc3e2t.NEF,http://193.205.194.113/RAISE/TIFF/r002fc3e2t.TIF,9/21/2014 3:29,5/4/2014 8:42,21.5 MB,L (4928 x 3264),42:11.6,"UTC+1, DST:ON",Lossless Compressed RAW (14-bit),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,outdoor; people
3,r00444b95t,http://193.205.194.113/RAISE/NEF/r00444b95t.NEF,http://193.205.194.113/RAISE/TIFF/r00444b95t.TIF,9/21/2014 12:20,9/2/2013 11:02,19.8 MB,L (4928 x 3264),02:23.5,"UTC+1, DST:ON",Lossless Compressed RAW (14-bit),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Manual,buildings; outdoor
4,r005f3e70t,http://193.205.194.113/RAISE/NEF/r005f3e70t.NEF,http://193.205.194.113/RAISE/TIFF/r005f3e70t.TIF,9/20/2014 23:19,2/18/2012 12:08,10.6 MB,L (4288 x 2848),08:05.0,"UTC+1, DST:OFF",Compressed RAW (12-bit),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,buildings; outdoor


In [6]:
links = df['TIFF'].iloc[:10].values

In [9]:
for i, link in enumerate(links):
    ! curl "$link" --output data/"$i".tif

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 14.1M  100 14.1M    0     0  12.2M      0  0:00:01  0:00:01 --:--:-- 12.2M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 19.6M  100 19.6M    0     0  14.3M      0  0:00:01  0:00:01 --:--:-- 14.3M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31.2M  100 31.2M    0     0  16.6M      0  0:00:01  0:00:01 --:--:-- 16.6M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 24.3M  100 24.3M    0     0  15.5M      0  0:00:01  0:00:01 --:--:-- 15.5M
  % Total    % Received % Xferd  Average Speed   Tim